In [1]:
from MFEA_lib.model import SMP_MFEA
from MFEA_lib.model.utils import *
from MFEA_lib.operators.Crossover import *
from MFEA_lib.operators.Mutation import *
from MFEA_lib.operators.Selection import *
from MFEA_lib.operators.Search import *
from MFEA_lib.tasks.Benchmark.Competitions import CEC17_benchmark, GECCO20_benchmark_50tasks, WCCI22_benchmark

In [2]:
def _updateProb(prob, k, dim_uss, nb_tasks, mean, std):
    for i in range(nb_tasks):
        for j in range(nb_tasks):
            kl = np.log((std[i] + 1e-50)/(std[j] + 1e-50)) + (std[j] ** 2 + (mean[j] - mean[i]) ** 2)/(2 * std[i] ** 2 + 1e-50) - 1/2
            # prob[i][j] = 1/(1 + kl/k)
            prob[i][j] = np.exp(-kl/k)

    return np.clip(prob, 1/dim_uss, 1)


In [22]:
import numpy as np
import numba
from numba import cuda

print(np.__version__)
print(numba.__version__)

cuda.detect()

1.23.4
0.56.3
Found 1 CUDA devices
id 0    b'NVIDIA GeForce GTX 1650'                              [SUPPORTED]
                      Compute Capability: 7.5
                           PCI Device ID: 0
                              PCI Bus ID: 1
                                    UUID: GPU-f3aef3b1-98ad-a6c1-74a1-210ce483e4ac
                                Watchdog: Enabled
             FP32/FP64 Performance Ratio: 32
Summary:
	1/1 devices are supported


True

In [25]:
from numba import cuda
print(cuda.gpus)

<Managed Device 0>


In [24]:
# Example 1.1: Add scalars
@cuda.jit
def add_array(a, b, c):
    i = cuda.threadIdx.x + cuda.blockDim.x * cuda.blockIdx.x
    if i < a.size:
        c[i] = a[i] + b[i]

N = 20
a = np.arange(N, dtype=np.float32)
b = np.arange(N, dtype=np.float32)
dev_c = cuda.device_array_like(a)

add_array[4, 8](a, b, dev_c)

c = dev_c.copy_to_host()
print(c)

[ 0.  2.  4.  6.  8. 10. 12. 14. 16. 18. 20. 22. 24. 26. 28. 30. 32. 34.
 36. 38.]


/home/tanminh/anaconda3/envs/lab/lib/python3.9/site-packages/numba/cuda/dispatcher.py:488: NumbaPerformanceWarning: Grid size 4 will likely result in GPU under-utilization due to low occupancy.
  warn(NumbaPerformanceWarning(msg))
/home/tanminh/anaconda3/envs/lab/lib/python3.9/site-packages/numba/cuda/cudadrv/devicearray.py:885: NumbaPerformanceWarning: Host array used in CUDA kernel will incur copy overhead to/from device.
  warn(NumbaPerformanceWarning(msg))


In [21]:
from numba import cuda, float32

# Controls threads per block and shared memory usage.
# The computation will be done on blocks of TPBxTPB elements.
TPB = 16

# @cuda.jit
# def fast_matmul(A, B, C):
#     # Define an array in the shared memory
#     # The size and type of the arrays must be known at compile time
#     sA = cuda.shared.array(shape=(TPB, TPB), dtype=float32)
#     sB = cuda.shared.array(shape=(TPB, TPB), dtype=float32)

#     x, y = cuda.grid(2)

#     tx = cuda.threadIdx.x
#     ty = cuda.threadIdx.y
#     bpg = cuda.gridDim.x    # blocks per grid

#     if x >= C.shape[0] and y >= C.shape[1]:
#         # Quit if (x, y) is outside of valid C boundary
#         return

#     # Each thread computes one element in the result matrix.
#     # The dot product is chunked into dot products of TPB-long vectors.
#     tmp = 0.
#     for i in range(bpg):
#         # Preload data into shared memory
#         sA[tx, ty] = A[x, ty + i * TPB]
#         sB[tx, ty] = B[tx + i * TPB, y]

#         # Wait until all threads finish preloading
#         cuda.syncthreads()

#         # Computes partial product on the shared memory
#         for j in range(TPB):
#             tmp += sA[tx, j] * sB[j, ty]

#         # Wait until all threads finish computing
#         cuda.syncthreads()

#     C[x, y] = tmp
@cuda.jit
def matmul(A, B, C):
    """Perform square matrix multiplication of C = A * B
    """
    i, j = cuda.grid(2)
    if i < C.shape[0] and j < C.shape[1]:
        tmp = 0.
        for k in range(A.shape[1]):
            tmp += A[i, k] * B[k, j]
        C[i, j] = tmp

from timeit import default_timer as timer  
A = np.random.rand(100, 100)
B = np.random.rand(100, 100)
C = np.random.rand(100, 100)
start = timer()
matmul(A, B, C)
print(timer() - start)

start = timer() 
fast_matmul(A, B, C)
print(timer() - start)

ValueError: 
Kernel launch configuration was not specified. Use the syntax:

kernel_function[blockspergrid, threadsperblock](arg0, arg1, ..., argn)

See https://numba.readthedocs.io/en/stable/cuda/kernels.html#kernel-invocation for help.



In [ ]:
@cuda.jit
# def fast_matmul(A, B, C):
#     # Define an array in the shared memory
#     # The size and type of the arrays must be known at compile time
#     sA = cuda.shared.array(shape=(TPB, TPB), dtype=float32)
#     sB = cuda.shared.array(shape=(TPB, TPB), dtype=float32)

#     x, y = cuda.grid(2)

#     tx = cuda.threadIdx.x
#     ty = cuda.threadIdx.y
#     bpg = cuda.gridDim.x    # blocks per grid

#     if x >= C.shape[0] and y >= C.shape[1]:
#         # Quit if (x, y) is outside of valid C boundary
#         return

#     # Each thread computes one element in the result matrix.
#     # The dot product is chunked into dot products of TPB-long vectors.
#     tmp = 0.
#     for i in range(bpg):
#         # Preload data into shared memory
#         sA[tx, ty] = A[x, ty + i * TPB]
#         sB[tx, ty] = B[tx + i * TPB, y]

#         # Wait until all threads finish preloading
#         cuda.syncthreads()

#         # Computes partial product on the shared memory
#         for j in range(TPB):
#             tmp += sA[tx, j] * sB[j, ty]

#         # Wait until all threads finish computing
#         cuda.syncthreads()

#     C[x, y] = tmp
@cuda.jit
def matmul(A, B, C):
    """Perform square matrix multiplication of C = A * B
    """
    i, j = cuda.grid(2)
    if i < C.shape[0] and j < C.shape[1]:
        tmp = 0.
        for k in range(A.shape[1]):
            tmp += A[i, k] * B[k, j]
        C[i, j] = tmp

from timeit import default_timer as timer  
A = np.random.rand(100, 100)
B = np.random.rand(100, 100)
C = np.random.rand(100, 100)
start = timer()
matmul(A, B, C)
print(timer() - start)

start = timer() 
fast_matmul(A, B, C)
print(timer() - start)

In [4]:
# cec17
t, ic = CEC17_benchmark.get_10tasks_benchmark()

ls_benchmark = [t]
ls_IndClass = [ic]
name_benchmark = ["cec17"]
path = './RESULTS/result/CEC17/SMP_v2/'

smpModel = MultiBenchmark(
    ls_benchmark= ls_benchmark,
    name_benchmark= name_benchmark,
    ls_IndClass= ls_IndClass,
    model= SMP_MFEA
)
smpModel.compile( 
    crossover = KL_SBXCrossover(nc= 2, k= 1000, conf_thres= 1 - 1e-12),
    # crossover = SBX_Crossover(nc = 2),
    mutation = PolynomialMutation(nm = 5, pm= 1),
    selection= ElitismSelection(random_percent= 0.1),
    search= L_SHADE(len_mem= 15),
    attr_tasks = ['crossover', 'mutation', 'search'],
)
smpModel.fit(
    nb_generations= 100, nb_inds_each_task= 1000, nb_inds_min= 200,
    lr = 0.1, p_const_intra= 0., prob_search = 0., lc_nums = 200,
    nb_epochs_stop= 1000, swap_po= False,
    evaluate_initial_skillFactor= True
)
a = smpModel.run(
    nb_run= 5,     
    save_path= path
)

'STT: 0'

Seed: 0 -- Time: 01m 3.93s  100 % [====================>]  Pop_size: 1.91E+03  ,  Cost: 2.54E+01  2.71E+01  3.87E+00  7.35E-02  3.48E+03  2.75E+00  3.24E+00  4.11E+01  2.14E-01  3.44E+02  ,  
[0.1 0.1 0.1 0.1 0.1 0.1 0.1 0.1 0.1 0.1]
[99884, 100742, 100064, 100332, 99528, 99566, 100036, 99812, 101122, 99864]
END!


'STT: 1'

Seed: 1 -- Time: 00m 57.84s  100 % [====================>]  Pop_size: 1.91E+03  ,  Cost: 2.92E+01  2.83E+01  3.84E+00  7.09E-02  3.34E+03  2.81E+00  3.15E+00  5.65E+01  2.68E-01  3.37E+02  ,  
[0.1 0.1 0.1 0.1 0.1 0.1 0.1 0.1 0.1 0.1]
[99944, 100184, 100356, 100484, 100268, 100030, 99252, 100130, 99988, 100314]
END!


'STT: 2'

Seed: 2 -- Time: 00m 57.56s  100 % [====================>]  Pop_size: 1.91E+03  ,  Cost: 3.64E+01  2.52E+01  4.51E+00  1.00E-01  3.20E+03  3.11E+00  3.33E+00  5.29E+01  3.00E-01  3.52E+02  ,  
[0.1 0.1 0.1 0.1 0.1 0.1 0.1 0.1 0.1 0.1]
[99530, 99918, 101306, 100414, 99852, 100538, 99882, 100014, 100204, 99292]
END!


'STT: 3'

Seed: 3 -- Time: 00m 44.48s   71 % [==============>     ]  Pop_size: 4.34E+03  ,  Cost: 1.07E+03  9.53E+02  7.59E+02  3.31E+00  1.11E+06  9.13E+00  2.05E+01  3.06E+03  1.20E+00  6.73E+02  ,  

/media/tanminh/New Volume/LinhTinh/Code_Lab/MFEA-MSOv2/MFEA_lib/model/utils/MultiTimeModel.py:83: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  result = np.array(result[:][:min([len(his) for his in result])][:])


AttributeError: 'model' object has no attribute 'last_pop'

In [5]:
model21 = loadModel("./RESULTS/result/GECCO20/LSA_2021/LSA_2021_MTOMSO_Benchmark_1.mso", ls_tasks=WCCI22_benchmark.get_50tasks_benchmark(1)[0])


FileNotFoundError: [Errno 2] No such file or directory: './RESULTS/result/GECCO20/LSA_2021/LSA_2021_MTOMSO_Benchmark_1.mso'

In [ ]:
len(model21.history_cost)

In [ ]:
model = loadModel("./RESULTS/result/GECCO20/LSA_2022/1.mso", ls_tasks=WCCI22_benchmark.get_50tasks_benchmark(1)[0])
model.set_attribute()

In [ ]:
len(model.history_cost)


In [ ]:
from MFEA_lib.model import AbstractModel, SMP_MFEA, MFEA_base, SMP_DE, LSA21
from MFEA_lib.model.utils import *
from MFEA_lib.model.utils import TuningModel
import pandas as pd
import numpy as np
import scipy.io
from MFEA_lib.tasks.Benchmark.Funcs import * 
from MFEA_lib.operators.Crossover import *
from MFEA_lib.operators.Mutation import *
from MFEA_lib.operators.Selection import *
from MFEA_lib.operators.Search import *

ls_benchmark = []
ls_IndClass = []
ls_tasks = [1,2,3,4,5,6,7,8,9,10]
name_benchmark = [] 

for i in ls_tasks:
    t, ic = WCCI22_benchmark.get_complex_benchmark(i)
    ls_benchmark.append(t)
    ls_IndClass.append(ic)
    name_benchmark.append(str(i))



smpModel = MultiBenchmark(
    ls_benchmark= ls_benchmark,
    name_benchmark= name_benchmark,
    ls_IndClass= ls_IndClass,
    model= LSA21
)

smpModel.compile(
    # crossover= newSBX(nc = 2, gamma= 0.6, alpha = 7),
    # crossover = SBX_LSA21(),
    # crossover = KL_SBX(nc= 2, k= 7),
    crossover= KL_SBX_LSA21(nc=2, k=7),
    # crossover = SBX_Crossover(nc=5),
    # mutation= GaussMutation(scale= 0.1),
    mutation = PolynomialMutation(pm=1, nm=7),
    # mutation = GMMScale(alpha = 3, lenMem= 20, default_scale= 0.5),
    selection= ElitismSelection(random_percent= 0.),
    search = LSHADE_LSA21(len_mem=30, p_ontop=0.11),
    attr_tasks = ['crossover', 'mutation', 'search'],
)
smpModel.fit(
    nb_generations= 1000, nb_inds_each_task= 100, nb_inds_min= 4,
    lr = 0.1
)
a = smpModel.run(
    nb_run= 2,
    save_path= './RESULTS/result/WCCI22_complex/SMP_v2/'
)

### Compare Benchmark

In [ ]:
compare = CompareResultBenchmark(path_folder="./RESULTS/result/GECCO20", ls_benchmark=[WCCI22_benchmark.get_50tasks_benchmark(i)[0] for i in range(1, 11)])

In [ ]:
compare.summarizing_compare_result(idx_main_algo=1, min_value=1e-6, idx_gener_compare=-1, total_generation=1000, combine=True)

In [ ]:
compare.show_compare_detail(idx_main_algo=1, min_value=1e-6, round=7,idx_gener_compare=-1, total_generation=1000) 
# idx_gener_compare : vị trí thế hệ mà tại đó muốn so sánh các thuật toán với nhau
# total generation : số thế hệ của thuật toán. mặc định là 1000 

In [ ]:
compare2 = CompareResultBenchmark(path_folder="./RESULTS/result/WCCI22_complex", ls_benchmark=[WCCI22_benchmark.get_complex_benchmark(i)[0] for i in range(1, 11)])

In [ ]:
compare2.summarizing_compare_result(idx_main_algo=0, min_value=1e-6, idx_gener_compare=-1, total_generation=1000)

In [ ]:
compare2.show_compare_detail(idx_main_algo=1, min_value=1e-6, round=7,idx_gener_compare=-1, total_generation=1000) 
# idx_gener_compare : vị trí thế hệ mà tại đó muốn so sánh các thuật toán với nhau
# total generation : số thế hệ của thuật toán. mặc định là 1000 

### Tuning Model

In [ ]:
ls_benchmark = [] 
ls_IndClass = [] 
name_benchmarks = []
for id in range(1, 11): 
    # b, i = WCCI22_benchmark.get_50tasks_benchmark(id) 
    b, i = WCCI22_benchmark.get_complex_benchmark(id) 
    name_benchmarks.append("50tasks_id_" + str(id))
    ls_benchmark.append(b) 
    ls_IndClass.append(i) 

benchmark_weights = np.array([1,1,1,1,1,1,1,1,1.0,1.0])
benchmark_weights /= np.sum(benchmark_weights)
benchmark_weights = benchmark_weights.tolist()

model = TuningModel(
    model_name= SMP_MFEA, 
    list_parameter= [
        # ('mutation', {
        #     'pm': [1, 1.5, 0.75],
        #     'nm': [5,7],
        # }),

        # ('lr', [0.05, 0.1,0.15]),
        ('prob_search', [1, 0.75, 0.5]),
        # ('nb_inds_min', [100, 50, 25])
    ],
)
model.compile(
    ls_benchmark = ls_benchmark, 
    benchmark_weights = benchmark_weights, 
    name_benchmark = name_benchmarks,
    ls_IndClass = ls_IndClass,

    # tasks = None,
    # IndClass= None,
    # tasks = CEC17_benchmark.get_10tasks_benchmark(),
    crossover= newSBX(nc = 2, gamma= 0.4, alpha = 6),
    # crossover= SBX_Crossover(nc = 2),
    # mutation = GaussMutation(scale = 0.01),
    mutation = PolynomialMutation(nm=5, pm= 1),
    selection= ElitismSelection(random_percent= 0.),
    search= SHADE(len_mem= 6, p_ontop= 0.1), 
    attr_tasks = ['crossover', 'mutation', 'search']
)

fit, com, result = model.run(
    path="./RESULTS/tuning_50/",
    replace_folder=True,
    min_value = 1e-6,
    
    nb_generations= 5, 
    nb_inds_each_task= 100, 
    nb_inds_min= 30,
    lr = 0.1,  
    p_const_intra= 0,  
    prob_search = 1.0,
    nb_epochs_stop= 1000, 
    swap_po= False,
    evaluate_initial_skillFactor= True
)

In [ ]:
ls_benchmark = [] 
ls_IndClass = [] 
for id in range(1, 3): 
    b, i = CEC17_benchmark.get_2tasks_benchmark(id) 
    ls_benchmark.append(b) 
    ls_IndClass.append(i) 

b, i = CEC17_benchmark.get_10tasks_benchmark() 
ls_benchmark.append(b)
ls_IndClass.append(i)

In [ ]:
model = TuningModel(
    model_name= SMP_MFEA, 
    list_parameter= [
        ('crossover', {
            'gamma': [0.4, 0.6]
        }), 
        ('lr', [0.04, 0.05]),
        ('crossover', {
            'alpha': [5, 7]
        }), 
    ],
)
model.compile(
    ls_benchmark = ls_benchmark, 
    benchmark_weights = (np.ones(shape= (3,)) / 3).tolist(), 
    name_benchmark = [str(i) for i in range(1,4)],
    ls_IndClass = ls_IndClass,

    # tasks = None,
    # IndClass= None,
    # tasks = CEC17_benchmark.get_10tasks_benchmark(),
    crossover= newSBX(nc = 2, gamma= 0.4, alpha = 6),
    # crossover= SBX_Crossover(nc = 2),
    mutation = GaussMutation(scale = 0.01),
    selection= ElitismSelection(random_percent= 0.),
    # search= SHADE(len_mem= 30, p_ontop= 0.1), 
    attr_tasks = ['crossover', 'mutation']
)

fit, com, result = model.run(
    replace_folder=True,
    min_value = 1e-6,
    
    nb_generations= 5, 
    nb_inds_each_task= 100, 
    nb_inds_min= 100,
    lr = 0.05,  
    p_const_intra= 0, 
    p_mutate= 0.1, 
    prob_search = 0.,
    nb_epochs_stop= 1000, 
    swap_po= False,
    evaluate_initial_skillFactor= True
)

### Show and compare result

In [ ]:
for id in range(1, 11): 
    SMP = loadModel("./RESULTS/__static__/GECCO20/SMP_NEWSBX_TUN_50/Gecco_id_"+ str(id) + ".mso", GECCO20_benchmark_50tasks.get_items(id)[0])
    SA = loadModel("./RESULTS/__static__/GECCO20/SA/SA_Benchmark_"+str(id) + ".mso", GECCO20_benchmark_50tasks.get_items(id)[0])
    LSA_21 = loadModel("./RESULTS/__static__/GECCO20/LSA_2021/LSA_2021_MTOMSO_Benchmark_"+str(id) + ".mso", GECCO20_benchmark_50tasks.get_items(id)[0])
    com = CompareModel([SMP, SA, LSA_21])
    print(com.detail_compare_result(min_value= 1e-6, round = 7))

In [ ]:
MFEA_model = loadModel('./RESULTS/MFEA_cec17.mso')
EBS_GA_model = loadModel('./RESULTS/EBS_GA_cec17.mso')
MaTDE_model = loadModel('./RESULTS/MaTDE_cec17.mso')
MaTGA_model = loadModel('./RESULTS/MaTGA_cec17.mso')
MTEA_AD_model = loadModel('./RESULTS/MTEA_AD_cec17.mso')
SA_model = loadModel('./RESULTS/SA_cec17.mso')
LSA_model = loadModel('./RESULTS/LSA_cec17.mso')
# SMP_NEWSBX = loadModel("./RESULTS/SMP_NEWSBX.mso", ls_tasks= CEC17_benchmark.get_10tasks_benchmark())


In [ ]:
compare = CompareModel(
    models = [MFEA_model, EBS_GA_model, MaTDE_model, MaTGA_model, MTEA_AD_model, SA_model, LSA_model],
    label= [...] * 7 ,
)
# compare.render(    
#     shape = (2, 5),
#     min_cost= 0,
#     step= 100,
#     yscale= 'log'
# )

In [ ]:
compare.detail_compare_result()

In [ ]:
compare.summarizing_compare_result(path=None , idx_main_algo= 2, nb_task= 2, ls_benchmark= [CEC17_benchmark.get_2tasks_benchmark(id) for id in range(1,10)])

In [ ]:
model = loadModel("./RESULTS/50tasks/SMP_NEWSBX/lan2_2s_newsbx_50task_1.mso")

In [ ]:
compare.summarizing_compare_result(path="./RESULTS/complex_2tasks/", idx_main_algo= 0, nb_task= 2, ls_benchmark= [WCCI22_benchmark.get_complex_benchmark(id)[0] for id in range(1,10)],min_value= 1e-6, combine= True)

In [ ]:
compare.summarizing_compare_result(path="./RESULTS/50tasks/", idx_main_algo= 7, nb_task= 50, ls_benchmark= [GECCO20_benchmark_50tasks.get_items(id) for id in range(1,11)],min_value= 1e-6, combine= False)

### Multi Time Model

In [ ]:
for id in range(4, 11): 
    tasks, IndClass= WCCI22_benchmark.get_50tasks_benchmark(id)

    smpModel = MultiTimeModel(model= SMP_MFEA)
    smpModel.compile(
        tasks= tasks,
        IndClass = IndClass,
        crossover= newSBX(nc = 2, gamma= 0.4, alpha = 6),
        mutation= GaussMutation(scale= 0.1),
        # mutation = GMMScale(alpha = 3, lenMem= 20, default_scale= 0.5),
        selection= ElitismSelection(random_percent= 0.),
        attr_tasks = ['crossover', 'mutation'],
        
    )
    smpModel.fit(
        nb_generations= 1000, nb_inds_each_task= 100, nb_inds_min= 30,
        lr = 0.15, p_const_intra= 0.0, p_mutate= 0.1,prob_search = 0.0,
        nb_epochs_stop= 1500, 
        evaluate_initial_skillFactor= True
    )
    smpModel.run(
        nb_run= 1,
        save_path= './RESULTS/results/smpv2_50tasks_id_'+ str(id) + '.mso'
    )

In [ ]:
for id in range(1, 11): 
    tasks, IndClass= WCCI22_benchmark.get_50tasks_benchmark(id)

    smpModel = MultiTimeModel(model= SMP_MFEA)
    smpModel.compile(
        tasks= tasks,
        IndClass = IndClass,
        crossover= newSBX(nc = 2, gamma= 0.4, alpha = 6),
        mutation= GaussMutation(scale= 0.1),
        # mutation = GMMScale(alpha = 3, lenMem= 20, default_scale= 0.5),
        selection= ElitismSelection(random_percent= 0.),
        attr_tasks = ['crossover', 'mutation'],
        
    )
    smpModel.fit(
        nb_generations= 1000, nb_inds_each_task= 100, nb_inds_min= 30,
        lr = 0.15, p_const_intra= 0.0, p_mutate= 0.1,prob_search = 0.0,
        nb_epochs_stop= 1500, 
        evaluate_initial_skillFactor= True
    )
    smpModel.run(
        nb_run= 5,
        save_path= './RESULTS/result/WCCI22_50tasks_id_'+ str(id) + '.mso'
    )

In [ ]:
tasks, IndClass= CEC17_benchmark.get_10tasks_benchmark()

smpModel = MultiTimeModel(model= SMP_MFEA)
smpModel.compile(
    tasks= tasks,
    IndClass = IndClass,
    crossover= newSBX(nc = 2, gamma= 0.4, alpha = 6),
    mutation= GaussMutation(scale= 0.1),
    # mutation = GMMScale(alpha = 3, lenMem= 20, default_scale= 0.5),
    selection= ElitismSelection(random_percent= 0.),
    search = SHADE(),
    attr_tasks = ['crossover', 'mutation'],
    
)
smpModel.fit(
    nb_generations= 10, nb_inds_each_task= 100, nb_inds_min= 30,
    lr = 0.15, p_const_intra= 0.0, p_mutate= 0.1,prob_search = 0.0,
    nb_epochs_stop= 1500, 
    evaluate_initial_skillFactor= True
)
smpModel.run(
    nb_run= 1,
    save_path= './RESULTS/result/MFEA_cec17_final.mso'
)

In [ ]:
smpModel.render_history()

In [ ]:
smpModel.print_result(['history_cost'])

In [ ]:
ls_benchmark = [] 
ls_IndClass = [] 
for id in range(1, 10): 
    b, i = CEC17_benchmark.get_2tasks_benchmark(id) 
    ls_benchmark.append(b) 
    ls_IndClass.append(i) 

b, i = CEC17_benchmark.get_10tasks_benchmark() 
ls_benchmark.append(b)
ls_IndClass.append(i)

smpModel = MultiBenchmark(
    ls_benchmark = ls_benchmark, 
    name_benchmark = [str(i) for i in range(1,4)],
    ls_IndClass = ls_IndClass,
    model= SMP_MFEA
    )

smpModel.compile(
    crossover= newSBX(nc = 2, gamma= 0.6, alpha = 7),
    mutation= GaussMutation(scale= 0.1),
    # mutation = GMMScale(alpha = 3, lenMem= 20, default_scale= 0.5),
    selection= ElitismSelection(random_percent= 0.),
    attr_tasks = ['crossover', 'mutation'],
    
)
smpModel.fit(
    nb_generations= 10, nb_inds_each_task= 100, nb_inds_min= 100,
    lr = 0.1, p_const_intra= 0.0, p_mutate= 0.1,prob_search = 0.0,
    nb_epochs_stop= 50, 
    evaluate_initial_skillFactor= True
)
smpModel.run(
    nb_run= 2,
    save_path= './RESULTS/result/smpMFEA_cec17.mso'
)

In [ ]:
smpModel.print_result(['history_cost'])

### SMP DE

In [ ]:
tasks, IndClass= WCCI22_benchmark.get_complex_benchmark(3)

smpModel = MultiTimeModel(model= SMP_MFEA)
smpModel.compile(
    tasks= tasks,
    IndClass = IndClass,
    crossover= newSBX(nc = 2, gamma= 0.6, alpha = 7),
    # crossover = SBX_Crossover(nc=5),
    # mutation= GaussMutation(scale= 0.1),
    mutation = PolynomialMutation(pm=1, nm=7),
    # mutation = GMMScale(alpha = 3, lenMem= 20, default_scale= 0.5),
    selection= ElitismSelection(random_percent= 0.),
    search = L_SHADE(len_mem=6, p_ontop=0.1),
    attr_tasks = ['crossover', 'mutation', 'search'],
    
)
smpModel.fit(
    nb_generations= 1000, nb_inds_each_task= 100, nb_inds_min= 30,
    lr = 0.1, p_const_intra= 0.0, p_mutate= 0.1,prob_search = 1.0,
    nb_epochs_stop= 1, 
    evaluate_initial_skillFactor= True, step_over= 5,
)
smpModel.run(
    nb_run= 1,
    save_path= './RESULTS/result/smpMFEA_cec17.mso'
)

In [ ]:
smpModel.render_smp()

In [ ]:
lsa_21 = loadModel("./RESULTS/__static__/GECCO20/LSA_2021/LSA_2021_MTOMSO_Benchmark_8.mso",WCCI22_benchmark.get_50tasks_benchmark(8)[0])
model = loadModel('./RESULTS/result/smpMFEA_cec17.mso', WCCI22_benchmark.get_50tasks_benchmark(8)[0])

compare = CompareModel([lsa_21, model])
compare.detail_compare_result(min_value=1e-6)

### SMP - LSHADE

In [ ]:
ls_benchmark = []
ls_IndClass = []

for i in range(1, 2, 1):
    t, ic = WCCI22_benchmark.get_complex_benchmark(i)
    ls_benchmark.append(t)
    ls_IndClass.append(ic)

name_benchmark = np.arange(len(ls_benchmark)) + 1

In [ ]:
ls_benchmark = []
ls_IndClass = []

for i in range(1, 3, 1):
    t, ic = WCCI22_benchmark.get_complex_benchmark(i)
    ls_benchmark.append(t)
    ls_IndClass.append(ic)

name_benchmark = np.arange(len(ls_benchmark)) + 1


smpModel = MultiBenchmark(
    ls_benchmark= ls_benchmark,
    name_benchmark= name_benchmark,
    ls_IndClass= ls_IndClass,
    model= SMP_MFEA
)

smpModel.compile(
    crossover= newSBX(nc = 2, gamma= 0.6, alpha = 7),
    # crossover = SBX_Crossover(nc=5),
    # mutation= GaussMutation(scale= 0.1),
    mutation = PolynomialMutation(pm=1, nm=7),
    # mutation = GMMScale(alpha = 3, lenMem= 20, default_scale= 0.5),
    selection= ElitismSelection(random_percent= 0.),
    search = L_SHADE(len_mem=6, p_ontop=0.1),
    attr_tasks = ['crossover', 'mutation', 'search'],
)
smpModel.fit(
    nb_generations= 5, nb_inds_each_task= 100, nb_inds_min= 30,
    lr = 0.1, p_const_intra= 0.0, prob_search = 1.0,
    nb_epochs_stop= 1000, swap_po= False,
    evaluate_initial_skillFactor= True
)
a = smpModel.run(
    nb_run= 1,
    save_path= './RESULTS/result/WCCI22_complex/SMP_v2/'
)


In [ ]:
model = loadModel("./RESULTS/result/WCCI22_complex/SMP_v2/1.mso", ls_tasks=WCCI22_benchmark.get_complex_benchmark(1)[0])

In [ ]:
model.history_cost[-1]

In [ ]:
compare = CompareResultBenchmark(
    path_folder= "./RESULTS/result/WCCI22_complex",
    ls_benchmark= [WCCI22_benchmark.get_complex_benchmark(i)[0] for i in range(1, 10)]
)

In [ ]:
compare.summarizing_compare_result()

In [ ]:
compare.show_compare_detail()

### Draft